In [1]:
# jesseLiu2000
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class TextCNN(nn.Module):
    def __init__(self, dropout=0.4):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(n_seq, embedding_size)
        self.cov1 = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels=embedding_size, out_channels=hidden_size, kernel_size=k),
                nn.LeakyReLU(),
                nn.MaxPool1d(kernel_size=k)
            ) for k in kernel_size
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size*len(kernel_size), n_class)

    def forward(self, X):
        embeded_X = self.embedding(X) # [batch_size, seq_length, embedding_size]
        embeded_X = embeded_X.permute(0, 2, 1)
        coved_X = [conv(embeded_X) for conv in self.cov1]
        coved_X = torch.cat(coved_X, dim=1) # [batch_size, hidden_size*len(kernel_size), 1]
        reshaped_X = torch.reshape(coved_X, [-1, hidden_size*len(kernel_size)]) # [batch_size, hidden_size*len(kernel_size)]
        dropout_X = self.dropout(reshaped_X)
        output = self.fc1(dropout_X) # [batch_size, num_classes]

        return output


if __name__ == '__main__':
    n_seq = 3
    embedding_size = 8
    hidden_size = 12
    kernel_size = [2, 2, 2]
    n_class = 2

    sentences = ['I hate milk', "I hate you", "sorry for that", "this is awful", 'I like cat', "I love you"]
    labels = [0, 0, 0, 0, 1, 1]

    word_vocab = ' '.join(sentences).split()
    word_vocab = list(set(word_vocab))
    word2index = {w: i for i, w in enumerate(word_vocab)}
    index2word = {i: w for i, w in enumerate(word_vocab)}

    n_seq = len(word_vocab)

    # inputs[batch_size, seq_length] => [6, 3]
    inputs = torch.LongTensor(np.asarray([np.asarray([word2index[word] for word in sent.split()]) for sent in sentences]))
    targets = torch.LongTensor(labels)

    model = TextCNN()

    lr = 1e-3
    epoch = 3000
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train
    for ep in range(epoch):
        optimizer.zero_grad()

        output = model(inputs)
        loss = criterion(output, targets)
        # print(output.size())

        if (ep + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (ep + 1), 'cost =', '{:.6f}'.format(loss))

        loss.backward()
        optimizer.step()

    # Test
    test_text = 'I hate you'
    tests = torch.LongTensor(np.asarray([np.asarray([word2index[n] for n in test_text.split()])]))

    # Predict
    predict = model(tests).data.max(1, keepdim=True)[1]
    if predict[0][0] == 0:
        print(test_text, "is Bad")
    else:
        print(test_text, "is Good")



Epoch: 1000 cost = 0.000067
Epoch: 2000 cost = 0.000370
Epoch: 3000 cost = 0.000002
I hate you is Bad
